In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from FRVRS import nu, fu
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [2]:

# load libraries
from datetime import timedelta
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

In [3]:

print([f'os.{fn}' for fn in dir(os) if 'dir' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'dir' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'github' in fn])

['os.DirEntry', 'os.O_DIRECT', 'os.O_DIRECTORY', 'os.ST_NODIRATIME', 'os.chdir', 'os.curdir', 'os.fchdir', 'os.listdir', 'os.makedirs', 'os.mkdir', 'os.pardir', 'os.removedirs', 'os.rmdir', 'os.scandir', 'os.supports_dir_fd']
['osp.curdir', 'osp.dirname', 'osp.isdir', 'osp.pardir']
['nu.github_folder']


In [4]:

print([f'os.{fn}' for fn in dir(os) if 'name' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'name' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'name' in fn])

['os.__name__', 'os.confstr_names', 'os.name', 'os.pathconf_names', 'os.rename', 'os.renames', 'os.sysconf_names', 'os.ttyname', 'os.uname', 'os.uname_result']
['osp.__name__', 'osp.basename', 'osp.dirname', 'osp.supports_unicode_filenames']
['nu.get_filename_from_url']


In [5]:

# Get the open world notebook names
file_ending = '.ipynb'
for sub_directory, directories_list, files_list in os.walk('../'):
    for file_name in files_list:
        if file_name.endswith(file_ending) and ('World' in file_name):
            file_path = osp.join(sub_directory, file_name)
            print(file_path)

../builders/Dataset Built for Metrics Evaluation Open World.ipynb
../builders/File Stats Created for Metrics Evaluation Open World.ipynb
../builders/Scene Stats Created for Metrics Evaluation Open World.ipynb
../explorations/Analyze Metrics Evaluation Open World Patient Stats.ipynb
../explorations/Metrics Evaluation Open World.ipynb
../maintenance/Data Fixes for Metrics Evaluation Open World.ipynb
../metrics/Correct SORT Order Metric for Metrics Evaluation Open World.ipynb


In [12]:

logs_path = "../data/logs/Metrics Evaluation Open World"
print()
def display_tree(path, level=0, is_last=False, indent="- ", pipe="+- ", corner=" "):
    """
    Displays directory tree structure with indentation and pipe characters.
    
    Parameters:
        path: Path to the directory.
        level: Current level of indentation (internal use).
        is_last: Whether the current directory is the last child (internal use).
        indent: Indentation string for folders except the last child (internal use).
        pipe: Pipe character to connect parent and child directories (internal use).
        corner: Corner character for the last child (internal use).
    """
    subdirs = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    for index, subdir in enumerate(subdirs):
        print((corner if is_last else pipe) * level + indent + os.path.basename(subdir))
        display_tree(subdir, level=level+1, is_last=(index==len(subdirs)-1), indent=indent, pipe=pipe, corner=corner)
try: display_tree(logs_path, indent="\t", pipe="\t", corner=" ")
except FileNotFoundError: print(f"Error: Path '{logs_path}' does not exist.")


	02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211
	04f80090-9e61-431d-8473-dccb75fed04d_2024207
	05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225
	0bc62b93-ac4d-40ef-bdfe-1d7badb24f70_2024208
	0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226
	1995e7ef-ef02-4fc1-b1ab-f137dbf69d48_2024217
	21f8cb5d-f5ac-4a01-9287-43df5f6751a1_2024219
	220b609b-0e35-454e-9afd-c84cbfa3e3ad_2024202
	23081f6e-875e-44f5-8bd0-edc3905f5c2c_2024201
	287389c4-4c48-4483-87c0-6b363b57bde2_2024220
	2e8f6555-a7fa-4b54-8132-c030d697b4ad_2024212
	37a554ee-fc49-4730-819c-2d97727bb0b7_2024218
	385032e9-9801-4dcf-a841-b3703a0d9acd_2024209
	3cf14e31-f416-4c78-8a69-91bf0c685448_2024215
	45365e18-6e38-48e7-b4a2-6b448b209034_2024218
	499179ba-3138-4bae-918e-ffc7fb943760_2024206
	4bc46c8c-66e7-463d-b3a1-2a8303af4fd1_2024203
	4de6e23f-3c00-49d9-bbac-3a9febab4311_2024222
	50b15e40-9860-4574-8ab8-0bd960fe27de_2024220
	5a909dc4-0f86-430d-a400-e37139cba69f_2024212
	5d8d73a3-1898-4f64-8676-73edd1b7daa0_2024211
	5d94f0d4-a1b1-4d18-8a62-591e1960

In [5]:

# Remove the prefix from the metrics notebook names
metrics_folder = '../metrics'
for old_notebook_name in os.listdir(metrics_folder):
    new_notebook_name = old_notebook_name.replace('Develop the ', '')
    os.rename(osp.join(metrics_folder, old_notebook_name), osp.join(metrics_folder, new_notebook_name))

In [4]:

os.listdir(path=None)

['Attic.ipynb', 'Installs.ipynb', 'OS Path Navigation.ipynb']

In [31]:

# Find out which fu methods, if any, are not being used
util_path = '../py/FRVRS/frvrs_utils.py'
utils_set = nu.get_utility_file_functions(util_path=util_path).difference(set(['__init__']))
ROGUE_FNS_DICT = {fn: 0 for fn in utils_set}
def add2dict(file_path, fn_regex):
    with open(file_path, 'r', encoding=nu.encoding_type) as f:
        lines_list = f.readlines()
        for line in lines_list:
            match_obj = fn_regex.search(line)
            if match_obj:
                fn = match_obj.group(1)
                ROGUE_FNS_DICT[fn] = ROGUE_FNS_DICT.get(fn, 0) + 1
for sub_directory, directories_list, files_list in os.walk(nu.github_folder):
    if all(map(lambda x: x not in sub_directory, ['.ipynb_checkpoints', '$Recycle.Bin'])):
        for file_name in files_list:
            if file_name.endswith('.ipynb') and not ('Attic' in file_name):
                file_path = osp.join(sub_directory, file_name)
                add2dict(file_path, re.compile(r'\bfu\.(' + '|'.join(utils_set) + r')\('))
add2dict(util_path, re.compile(r'\bself\.(' + '|'.join(utils_set) + r')\('))
df = DataFrame.from_dict(ROGUE_FNS_DICT, orient='index', columns=['use_count'])
mask_series = (df.use_count == 0)
print(df[mask_series].index.tolist())
df[~mask_series].sort_values('use_count')

['get_still_patients', 'get_dead_patients', 'get_patient_location', 'get_is_duplicate_file', 'get_triage_priority_data_frame', 'split_df_by_teleport', 'is_patient_still', 'get_player_location']


,use_count
is_patient_gazed_at,1
get_tag_value,1
set_scene_indices,1
get_pulse_value,1
get_injury_correctly_treated_time,1
...,...
format_timedelta,26
get_scene_type,31
get_is_scene_aborted,33
get_logger_version,33


In [ ]:

nu.open_path_in_notepad('~/anaconda3/lib/python3.11/calendar.py', verbose=True)

In [ ]:

# Define the starting directory
root_dir = '/'

# Use a recursive function to search for matching folders
def find_python311_folders(directory):
    found_folders = []
    try: entries_list = os.scandir(directory)
    except: entries_list = []
    for entry in entries_list:
        try: is_entry_dir = entry.is_dir()
        except: is_entry_dir = False
        if is_entry_dir:
            
            # Check if the current entry is the target folder
            if entry.name == "python3.11": found_folders.append(os.path.join(directory, entry.name))
            
            # Recursively search subdirectories (except the target folder)
            elif entry.name != "python3.11": found_folders.extend(find_python311_folders(os.path.join(directory, entry.name)))
    
    return found_folders

# Start the search from the root directory
python311_folders = find_python311_folders(root_dir)

In [ ]:

# Print the found folders
print(f"Found python3.11 folders: {', '.join(python311_folders)}")

In [ ]:

# Print out a section of the .gitignore
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git']
print()
print('# exclude everything except personal directories')
print('/*')
print('!.gitignore')
for sub_directory in os.listdir(nu.github_folder):
    if all(map(lambda x: x not in sub_directory, black_list)) and osp.isdir(osp.join(nu.github_folder, sub_directory)): print(f'!/{sub_directory}')
print('!README.md')

In [ ]:

columns_set = set()
for pickle_name in [fn.split('.')[0] for fn in os.listdir(s.saves_pickle_folder)]:
    if pickle_name.endswith('_df'):
        df = nu.load_object(pickle_name)
        columns_set.update(df.columns[list(map(lambda x: ('year' in x) and (('begin' in x) or ('end' in x)), df.columns))])
columns_set

In [ ]:

# 'Intel64 Family 6 Model 60 Stepping 3, GenuineIntel'
import platform

platform.processor()